In [1]:
from datetime import date
from glob import glob
import json
import os
import math
import sys
import time

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
import gspread
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp
from tbd_eeg.tbd_eeg.data_analysis.Utilities.utilities import get_stim_events, get_evoked_traces, find_nearest_ind

In [5]:
%matplotlib notebook

#### Functions

In [6]:
def get_stim_event_inds(stim_table, stim_type, stim_param, sweep, trials='stationary'):
    if trials == 'resting':
        return stim_table[
            (stim_table['stim_type'] == stim_type) &
            (stim_table['parameter'] == stim_param) &
            (stim_table['sweep'] == sweep) &
            (stim_table['good'] == True) &
            (stim_table['resting_trial'] == True)
        ].index.values
    elif trials == 'running':
        return stim_table[
            (stim_table['stim_type'] == stim_type) &
            (stim_table['parameter'] == stim_param) &
            (stim_table['sweep'] == sweep) &
            (stim_table['good'] == True) &
            (stim_table['resting_trial'] == False)
        ].index.values
    else:
        return stim_table[
            (stim_table['stim_type'] == stim_type) &
            (stim_table['parameter'] == stim_param) &
            (stim_table['sweep'] == sweep) &
            (stim_table['good'] == True)
        ].index.values

In [7]:
def p_stars(test_pval):
    if test_pval < 0.001:
        return '***'
    elif test_pval < 0.01:
        return '**'
    elif test_pval < 0.05:
        return '*'
    else:
        return 'n.s.'

#### Load Zap_Zip-log_exp to get metadata for experiments

In [8]:
_gc = gspread.service_account() # need a key file to access the account
_sh = _gc.open('Zap_Zip-log_exp') # open the spreadsheet
_df = pd.DataFrame(_sh.sheet1.get()) # load the first worksheet
zzmetadata = _df.T.set_index(0).T # put it in a nicely formatted dataframe

In [9]:
state_colors = {
    'awake': (120/255, 156/255, 74/255),
    'anesthetized': (130/255, 122/255, 163/255),
    'recovery': (93/255, 167/255, 229/255)
}
c_vermillion = (213/255, 94/255, 0/255)
c_bgreen = (0/255, 158/255, 115/255)
c_skyblue = (86/255, 180/255, 233/255)

### Load subjects list from .json file

In [10]:
with open(r'C:\Users\lesliec\OneDrive - Allen Institute\data\all_iso_subjects.json') as subjects_file:
    multi_sub_dict = json.load(subjects_file)

In [11]:
for group, group_subs in multi_sub_dict.items():
    print(group)
    print('')
    for mouse_num, mdata in group_subs.items():
        print(' {}'.format(mouse_num))
        mdata['exp'] = EEGexp(mdata['data_loc'], preprocess=False, make_stim_csv=False)
        print('')

MOs_superficial

 521885
Experiment type: electrical stimulation

 521886
Experiment type: electrical stimulation

 521887
Experiment type: electrical stimulation

 543393
Experiment type: electrical stimulation

 543394
Experiment type: electrical stimulation

 575102
Experiment type: electrical and sensory stimulation

 571619
Experiment type: electrical stimulation

MOs_deep

 546655
Experiment type: electrical and sensory stimulation

 551399
Experiment type: electrical stimulation

 551397
Experiment type: electrical and sensory stimulation

 569062
Experiment type: electrical and sensory stimulation

 569068
Experiment type: electrical and sensory stimulation

 569069
Experiment type: electrical and sensory stimulation

 569064
Experiment type: electrical and sensory stimulation

 569073
Experiment type: electrical and sensory stimulation

 571619
Experiment type: electrical stimulation

 569070
Experiment type: electrical stimulation

SSp_superficial

 571620
Experiment type: el

In [12]:
savedir = r'C:\Users\lesliec\OneDrive - Allen Institute\data'

## Find average isoflurane induction time

In [15]:
start = time.time()
all_iso_inductions = []
for group, group_subs in multi_sub_dict.items():
    print(group)
    for mouse_num, mdata in group_subs.items():
        try:
            iso_ind, iso_maint = mdata['exp'].load_iso_times()
            print(' {}: {:.2f} s'.format(mouse_num, np.diff(iso_ind)[0]))
            all_iso_inductions.append(np.diff(iso_ind)[0] / 60)
        except IndexError:
            print(' {}: cannot get iso times'.format(mouse_num))
        
    print('')
end = time.time()
print('Time to load all subjects: {:.2f} mins'.format((end-start)/60))

MOs_superficial
 521885: cannot get iso times
 521886: 215.97 s
 521887: 108.63 s
 543393: 269.57 s
 543394: 414.74 s
 575102: 215.16 s
 571619: 133.18 s

MOs_deep
 546655: 355.96 s
 551399: 273.16 s
 551397: 239.97 s
 569062: 257.97 s
 569068: 537.93 s
 569069: 279.16 s
 569064: 199.57 s
 569073: 509.54 s
 571619: 133.18 s
 569070: 290.36 s

SSp_superficial
 571620: 225.96 s
 586466: 211.17 s
 590479: 210.37 s
 590480: 135.17 s
 599017: 237.56 s

SSp_deep
 569073: 81.19 s
 569072: 223.16 s
 571620: 225.96 s
 586466: 211.17 s
 586468: 125.97 s
 590479: 210.37 s
 590480: 135.17 s
 599017: 237.56 s

Time to load all subjects: 0.48 mins


In [17]:
print('Mean induction time: {:.1f} min'.format(np.mean(all_iso_inductions)))
print('SEM: {:.1f} min'.format(np.std(all_iso_inductions)/(np.sqrt(len(all_iso_inductions)))))

Mean induction time: 4.0 min
SEM: 0.3 min


In [15]:
## Remove duplicate subjects (subjects with two depths) ##
all_subs_ind = np.unique(all_iso_inductions)

## Remove outliers ##
rem_outs = all_subs_ind[all_subs_ind < 600]

In [16]:
print('Mean induction time: {:.2f} min'.format(np.mean(rem_outs)/60))
print('SEM: {:.2f} min'.format(np.std(rem_outs)/(np.sqrt(len(rem_outs))*60)))

Mean induction time: 4.06 min
SEM: 0.36 min


## Find location of stim electrode

In [17]:
zzmetadata.head()

,mouse_name,exp_name,brain states,stimulation,visual_stim,audio_stim,ISI (sec),stimulus duration (msec),Current (uA),Cortical Area stimulation,N trials per stimulus,EEG bad_channels,Npx,Units Sorted (X),Brain slices (X),,Brain areas assignment,"CCF coordinates stim electrode (surface,tip)","CCF area stim electrode (surface,tip)",Notes
1,mouse496220,audio_vis1_2020-06-10_14-54-43,awake/ISO,sensory,black/white,whitenoise/10000,5,250,,,60,None,None,None,None,None,None,None,None,None
2,mouse496220,audio_vis2_2020-06-11_11-42-47,awake/ISO,sensory,black/white,whitenoise/10000,5,250,,,60,29,None,None,None,None,None,None,None,None
3,mouse496220,audio_vis3_2020-06-16_10-35-57,run/resting,sensory,black,whitenoise,5,250,,,20,None,None,None,None,None,None,None,None,None
4,mouse496220,audio_vis4_2020-06-18_13-49-17,run/resting,sensory,black/white,whitenoise/10000,5,250,,,60,None,None,None,None,None,None,None,None,None
5,mouse521885,audio_vis1_2020-07-08_12-37-58,awake/ISO,sensory,black/white,whitenoise/10000,[3.5 4.5],250,,,50,6,None,None,None,None,None,None,None,None


In [18]:
zzmetadata[5:10]

,mouse_name,exp_name,brain states,stimulation,visual_stim,audio_stim,ISI (sec),stimulus duration (msec),Current (uA),Cortical Area stimulation,N trials per stimulus,EEG bad_channels,Npx,Units Sorted (X),Brain slices (X),,Brain areas assignment,"CCF coordinates stim electrode (surface,tip)","CCF area stim electrode (surface,tip)",Notes
6,mouse521885,estim1_2020-07-09_14-23-49,awake/ISO,electrical,N/A,N/A,[3.5 4.5],0.2,20/50/100,M2,60,"6,7,8,10,11,12,13,14,15,16,17,18,21",None,None,None,None,None,None,None,None
7,mouse521886,audio_vis1_2020-07-15_13-28-29,awake/ISO,sensory,black/white,whitenoise/10000,[3.5 4.5],250,,,60,None,None,None,None,None,None,None,None,None
8,mouse521886,estim1_2020-07-16_13-37-02,awake/ISO/recovery,electrical,N/A,N/A,[3.5 4.5],0.2,20/50/100,M2,100,"10,11,12,13,14,15,16,17,18,19",None,None,None,None,None,None,None,None
9,mouse521887,audio1_2020-07-29_09-13-05,awake/awake/awake/ISO/ISO/ISO_low/recovery/rec...,sensory,N/A,whitenoise/10000,2.5,250,,,100,"7,18",None,None,None,None,None,None,None,None
10,mouse521887,estim1_2020-07-30_11-25-05,awake/awake,electrical,N/A,N/A,[3.5 4.5],0.2,20,M2,100,"4,7,9,11,12,13,14,15,18",None,None,None,None,None,None,None,None


### Test multiple subjects

In [20]:
all_stim_locs = []
for group, group_subs in multi_sub_dict.items():
    print(group)
    stim_area = group[:2]
    stim_depth = group.split('_')[-1]
    for mouse_num, mdata in group_subs.items():
        exp_meta = zzmetadata[(
            (zzmetadata['mouse_name'].str.contains(mdata['exp'].mouse)) &
            (zzmetadata['exp_name'].str.contains(os.path.basename(os.path.dirname(mdata['exp'].experiment_folder))))
        )].squeeze()
        
        if exp_meta['Brain slices (X)'] == 'X':
#             print(' {} brain slices'.format(mouse_num))
            mmpix = 0.01 # mm, histology used CCF with 10 um resolution
            
        elif exp_meta['Brain slices (X)'] == 'Tissuecyte':
#             print(' {} tissuecyte'.format(mouse_num))
            mmpix = 0.025 # mm, tissucyte used CCF with 25 um resolution
            
        else:
            print(' {} has no histology info'.format(mouse_num))
            all_stim_locs.append([
                group, mouse_num, stim_area, stim_depth, np.nan
            ])
            continue
            
        if exp_meta['CCF coordinates stim electrode (surface,tip)'][0] != '[':
            print(' {} no stim location, skipping'.format(mouse_num))
            all_stim_locs.append([
                group, mouse_num, stim_area, stim_depth, np.nan
            ])
            continue
            
        coords_char = exp_meta['CCF coordinates stim electrode (surface,tip)'].replace(' ','')
        all_coords = []
        for charlong in coords_char.split(']'):
            coordstemp = []
            for chara in charlong.replace('[','').split(','):
                if chara.isdecimal():
                    coordstemp.append(int(chara))
            if len(coordstemp) > 0:
                all_coords.append(coordstemp)

        surface_coords = np.array(all_coords[0])
        tip_coords = np.array(all_coords[1])
        stim_depth_mm = (tip_coords[1] - surface_coords[1]) * mmpix
        print(' {}: {:.2f} mm'.format(mouse_num, stim_depth_mm))
        
        all_stim_locs.append([
            group, mouse_num, stim_area, stim_depth, stim_depth_mm
        ])
          
    print('')
sub_stim_locs = pd.DataFrame(all_stim_locs, columns=['group', 'mouse', 'stim_cortex', 'stim_depth', 'depth_mm'])

MOs_superficial
 521885 has no histology info
 521886 has no histology info
 521887 has no histology info
 543393 has no histology info
 543394 has no histology info
 575102: 0.65 mm
 571619: 1.42 mm

MOs_deep
 546655: 1.25 mm
 551399: 1.12 mm
 551397: 1.24 mm
 569062: 1.54 mm
 569068: 1.53 mm
 569069: 1.10 mm
 569064: 1.38 mm
 569073: 1.45 mm
 571619: 1.42 mm
 569070 no stim location, skipping

SSp_superficial
 571620: 0.95 mm
 586466: 1.34 mm
 590479: 0.93 mm
 590480: 1.05 mm
 599017: 1.05 mm

SSp_deep
 569073 no stim location, skipping
 569072 no stim location, skipping
 571620: 0.95 mm
 586466: 1.34 mm
 590479: 0.93 mm
 590480: 1.05 mm
 599017: 1.05 mm



In [22]:
sub_stim_locs.tail()

,group,mouse,stim_cortex,stim_depth,depth_mm
24,SSp_deep,571620,SS,deep,0.950
25,SSp_deep,586466,SS,deep,1.340
26,SSp_deep,590479,SS,deep,0.925
27,SSp_deep,590480,SS,deep,1.050
28,SSp_deep,599017,SS,deep,1.050


Save it to .csv

TEST

In [29]:
group = 'SSp_deep'
mouse_num = '569072'
mdata = multi_sub_dict[group][mouse_num]

exp_meta = zzmetadata[(
    (zzmetadata['mouse_name'].str.contains(mdata['exp'].mouse)) &
    (zzmetadata['exp_name'].str.contains(os.path.basename(os.path.dirname(mdata['exp'].experiment_folder))))
)].squeeze()

In [30]:
exp_meta

0
mouse_name                                                                            mouse569072
exp_name                                                            estim_vis_2021-04-22_10-26-58
brain states                                                                            awake/ISO
stimulation                                                                    electrical/sensory
visual_stim                                                                                 white
audio_stim                                                                                    N/A
ISI (sec)                                                                               [3.5 4.5]
stimulus duration (msec)                                                                  0.2/250
Current (uA)                                                                             30/50/70
Cortical Area stimulation                                                               SS-cortex
N trials per stimu

In [77]:
if exp_meta['Brain slices (X)'] == 'X':
    print('brain slices')
    mmpix = 0.01 # mm, histology used CCF with 10 um resolution
elif exp_meta['Brain slices (X)'] == 'Tissuecyte':
    print('tissuecyte')
    mmpix = 0.025 # mm, tissucyte used CCF with 25 um resolution
else:
    print('no histology info')

brain slices


In [78]:
exp_meta['CCF coordinates stim electrode (surface,tip)']

'[444,111,417], [434,176,434]'

In [79]:
coords_char = exp_meta['CCF coordinates stim electrode (surface,tip)'].replace(' ','')
all_coords = []
for charlong in coords_char.split(']'):
    coordstemp = []
    for chara in charlong.replace('[','').split(','):
        if chara.isdecimal():
            coordstemp.append(int(chara))
    if len(coordstemp) > 0:
        all_coords.append(coordstemp)

In [80]:
all_coords

[[444, 111, 417], [434, 176, 434]]

In [81]:
surface_coords = np.array(all_coords[0])
tip_coords = np.array(all_coords[1])

len_stim_elec = np.linalg.norm(surface_coords - tip_coords) * mmpix
print(len_stim_elec)

0.6792643079096677


In [82]:
print((tip_coords[1] - surface_coords[1]) * mmpix)

0.65


In [83]:
## CCF and annotated volume ##
mcc = MouseConnectivityCache(resolution=int(mmpix * 1E3))
annot, annot_info = mcc.get_annotation_volume()
structure_tree = mcc.get_structure_tree()

In [84]:
surf_pix = np.nonzero(annot[tip_coords[0],:,tip_coords[2]])[0][0]
print(surf_pix)

108


In [85]:
print((tip_coords[1] - surf_pix) * mmpix)

0.68
